<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/summarization/T5_large_Finetune_multi_news_summarization_v2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summarization

creates a shorter version of a document or an article that captures all the important information. Along with translation, it is another example of a task that can be formulated as a sequence-to-sequence task. Summarization can be:

- Extractive: extract the most relevant information from a document.
- Abstractive: generate new text that captures the most relevant information.

https://huggingface.co/docs/transformers/tasks/summarization



### Dataset --> multi_news dataset for summarization
Multi-News, consists of news articles and human-written summaries of these articles from the site newser.com. Each summary is professionally written by editors and includes links to the original articles cited.

There are two features:

document: text of news articles seperated by special token "|||||".
summary: news summary.


@misc{alex2019multinews,
    title={Multi-News: a Large-Scale Multi-Document Summarization Dataset and Abstractive Hierarchical Model},
    author={Alexander R. Fabbri and Irene Li and Tianwei She and Suyi Li and Dragomir R. Radev},
    year={2019},
    eprint={1906.01749},
    archivePrefix={arXiv},
    primaryClass={cs.CL}
}


other Datasets

https://huggingface.co/datasets/ccdv/pubmed-summarization

https://huggingface.co/datasets/samsum


In [1]:
# Transformers installation
! pip install -q --disable-pip-version-check py7zr sentencepiece loralib peft trl
! pip install -q    wandb bitsandbytes
! pip install datasets evaluate rouge_score -q
! pip install transformers[torch] -q
! pip install accelerate -U -q
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



1. Finetune [T5](https://huggingface.co/t5-base) on the Multi-News [multi_news](https://huggingface.co/datasets/multi_news) dataset for abstractive summarization.
2. Use your finetuned model for inference.

<Tip>
Model architectures:

<!--This tip is automatically generated by `make fix-copies`, do not fill manually!-->

[BART](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bart), [BigBird-Pegasus](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bigbird_pegasus), [Blenderbot](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/blenderbot), [BlenderbotSmall](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/blenderbot-small), [Encoder decoder](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/encoder-decoder), [FairSeq Machine-Translation](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/fsmt), [GPTSAN-japanese](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gptsan-japanese), [LED](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/led), [LongT5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/longt5), [M2M100](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/m2m_100), [Marian](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/marian), [mBART](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mbart), [MT5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mt5), [MVP](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mvp), [NLLB](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nllb), [NLLB-MOE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nllb-moe), [Pegasus](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/pegasus), [PEGASUS-X](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/pegasus_x), [PLBart](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/plbart), [ProphetNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/prophetnet), [SwitchTransformers](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/switch_transformers), [T5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/t5), [XLM-ProphetNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm-prophetnet)

<!--End of the generated tip-->


In [3]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
from torch import cuda, bfloat16
import transformers

import torch
import torch.nn as nn
from google.colab import userdata
import wandb

In [4]:
from google.colab import output
output.enable_custom_widget_manager()

In [5]:
PROJECT = "T5-large-Summarization"
MODEL_NAME = "google-t5/t5-large"
DATASET = "multi_news"

In [6]:


wandb_key = userdata.get('WANDB')
wandb.login(key=wandb_key)

wandb.init(project=PROJECT, # the project I am working on
           tags=[MODEL_NAME, DATASET],
           notes ="Fine tuning T5 large with ccdv/pubmed-summarization Dataset. Text Summarization") # the Hyperparameters I want to keep track of

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: olonok (olonok69). Use `wandb login --relogin` to force relogin


In [7]:
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device


'cuda:0'

## Load multi_news dataset

https://huggingface.co/datasets/multi_news

In [8]:
from datasets import load_dataset

dataset  = load_dataset("multi_news")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Split the dataset into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [10]:
print(f"Train dataset size: {len(dataset['train'])}")
print(f"test dataset size: {len(dataset['test'])}")
print(f"Validation dataset size: {len(dataset['validation'])}")

Train dataset size: 44972
test dataset size: 5622
Validation dataset size: 5622


Then take a look at an example:

In [11]:
dataset['train'][100]['document']

'Katy Perry is all about breaking conventional beauty rules, from her love of everything technicolor and coated in glitter, to her no-brows, black lipstick Met Gala look. So, of course, the pop star — and face of CoverGirl — was the perfect person to help announce that the beauty brand has named its first-ever male CoverGirl, social media star James Charles. \n \n According to a press release from the brand, all CoverGirls “are role models and boundary-breakers, fearlessly expressing themselves, standing up for what they believe, and redefining what it means to be beautiful,” and who better to embody that ethos than Instagram sensation James Charles. After launching his beauty account a year ago, the teen has since quickly attracted hundreds of thousands of followers (427,000 to be exact) thanks to his unique, transformative approach to makeup artistry. \n \n RELATED PHOTOS: Katy Perry’s Most Outrageous Twitpics \n \n While Charles’ partnership with the brand kicks off today, we’ll hav

In [12]:
len(dataset['train'][100]['document'])

6217

In [13]:
dataset['train'][100]['summary']

'– If a woman can be president, who\'s to say a man can\'t be a CoverGirl. On Tuesday, the makeup company\'s current spokesperson, Katy Perry, announced James Charles as the first ever "CoverBoy" on her Instagram page. Charles, a 17-year-old "aspiring makeup artist," started using makeup only a year ago but has already amassed more than 430,000 followers on Instagram, the Huffington Post reports. According to People, Charles will appear in TV, print, and digital ads for "So Lashy" mascara later this month and will work with CoverGirl through 2017. "I am so thankful and excited," Charles posted on Instagram. "And yes I know I have lipstick on my teeth. It was a looonnnnggg day." CoverGirl says it wants to work with "role models and boundary-breakers, fearlessly expressing themselves, standing up for what they believe, and redefining what it means to be beautiful," Teen Vogue reports. The company calls Charles an inspiration. Teen Vogue is definitely on board, stating: "We\'re firm belie

In [14]:
len(dataset['train'][100]['summary'])

1268

There are two fields that you'll want to use:

- `text`: the text of the bill which'll be the input to the model.
- `summary`: a condensed version of `text` which'll be the model target.

## Preprocess

The next step is to load a T5 tokenizer to process `text` and `summary`:

Model--> https://huggingface.co/google-t5/t5-large

In [15]:
from transformers import AutoTokenizer

checkpoint = model_id = MODEL_NAME
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

The preprocessing function you want to create needs to:

1. Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2. Use the keyword `text_target` argument when tokenizing labels.
3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [16]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/44972 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) method. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [18]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5622
    })
})

In [19]:
len(tokenized_dataset['train'][100]['labels']), len(tokenized_dataset['train'][100]['input_ids'])

(256, 1024)

## Train

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load T5 with [AutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSeq2SeqLM):

At this point, only three steps remain:

1. Define your training hyperparameters in [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the ROUGE metric and save the training checkpoint.
2. Pass the training arguments to [Seq2SeqTrainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [20]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [21]:
def print_number_of_trainable_model_parameters(model, tag="original_model", to_wandb=False):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()

    if to_wandb:
      wandb.log({f'{tag}': {"trainable_model_params":trainable_model_params}})
      wandb.log({f'{tag}': {"all_model_params":all_model_params}})
      wandb.log({f'{tag}': {"percentage_of_trainable_model_parameters": 100 * trainable_model_params}} )

    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params}%"

In [22]:
repository_id = f"{checkpoint.split('/')[1]}-{DATASET}"
repository_id

't5-large-multi_news'

In [23]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
dataset_id = "multi_news"
# Hugging Face repository id
repository_id = f"{checkpoint.split('/')[1]}-{DATASET}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=10,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="wandb",
)



Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

In [24]:
print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 737668096
all model parameters: 737668096
percentage of trainable model parameters: 100.0%


In [25]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
        label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8

)

In [26]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

141

In [27]:


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,

)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [28]:

with wandb.init(project=PROJECT, job_type="train", # the project I am working on
           tags=[MODEL_NAME, DATASET],
           notes =f"Fine tuning {MODEL_NAME} with {DATASET}. Summarization Documents"):

  print_number_of_trainable_model_parameters(model,"original_model",to_wandb=True)

  trainer.train()

Epoch,Training Loss,Validation Loss
1,2.299900,2.131090
2,2.137600,2.072646
3,2.030500,2.060889
4,1.977100,2.057110
5,1.938900,2.055043
6,1.881600,2.055076
7,1.848400,2.059933


Epoch,Training Loss,Validation Loss
1,2.299900,2.131090
2,2.137600,2.072646
3,2.030500,2.060889
4,1.977100,2.057110
5,1.938900,2.055043
6,1.881600,2.055076
7,1.848400,2.059933
8,1.824800,2.060389
9,1.830600,2.062700
10,1.818200,2.062733


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


eval/loss,█▃▂▁▁▁▁▁▂▂
eval/runtime,▄▄▆▃▂▂▂▁██
eval/samples_per_second,▅▅▃▆▇▇▇█▁▁
eval/steps_per_second,▅▅▃▆▇▇▇█▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▆▅▅▃▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [29]:
trainer.push_to_hub("olonok/t5-large-multi_news-summarization")

model.safetensors:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/olonok/t5-large-multi_news/commit/d59755c99f629bdbc865a8d97623e8b3d592100d', commit_message='olonok/t5-large-multi_news-summarization', commit_description='', oid='d59755c99f629bdbc865a8d97623e8b3d592100d', pr_url=None, pr_revision=None, pr_num=None)

<Tip>

For a more in-depth example of how to finetune a model for summarization, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization.ipynb)
or [TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization-tf.ipynb).

</Tip>

In [30]:

! rm -rf t5-sum-checkpoint

In [31]:

!mkdir t5-sum-checkpoint
custom_path = "./t5-sum-checkpoint/"
trainer.save_model(output_dir=custom_path)

In [33]:

with wandb.init(project=PROJECT, job_type="models"):
  artifact = wandb.Artifact("T5-large_Summarization_model", type="model")
  artifact.add_dir(custom_path)
  wandb.save(custom_path)
  wandb.log_artifact(artifact)


wandb: Adding directory to artifact (./t5-sum-checkpoint)... Done. 4.2s


## Inference

Great, now that you've finetuned a model, you can use it for inference!

Come up with some text you'd like to summarize. For T5, you need to prefix your input depending on the task you're working on. For summarization you should prefix your input as shown below:

In [28]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [29]:
run = wandb.init()

artifact = run.use_artifact('olonok69/T5-large-Summarization/T5-large_Summarization_model:v0', type='model')
artifact_dir = artifact.download()

fine_tune_model=  AutoModelForSeq2SeqLM.from_pretrained(artifact_dir, torch_dtype=torch.bfloat16)

wandb: Downloading large artifact T5-large_Summarization_model:v0, 1407.06MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:1:22.7


In [30]:
artifact_dir

'/content/artifacts/T5-large_Summarization_model:v0'

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for summarization with your model, and pass your text to it:

In [31]:
from transformers import pipeline

In [32]:


summarizer = pipeline("summarization", model=fine_tune_model, tokenizer=tokenizer)
response = summarizer(text)

Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


In [33]:
response[0]['summary_text']

'– Inflation isn\'t just a global problem, it\'s a national one, and President Obama\'s plan to cut the deficit and raise taxes on the rich and the super-wealthy is a bold one. "It\'s the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country," he writes in the Washington Post. "And no one making under $400,000 per year will pay a penny more in taxes." The plan, he says, will lower energy costs, lower prescription drug costs, and lower the deficit.'

In [34]:
checkpoint

'google-t5/t5-large'

In [35]:
from transformers import pipeline

summarizer_ori = pipeline("summarization", model=checkpoint, tokenizer=tokenizer)
response_ori = summarizer_ori(text)

Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


In [36]:
response_ori[0]['summary_text']

"the inflation reduction act lowers prescription drug costs, health care costs, and energy costs . it's the most aggressive action on tackling the climate crisis in american history . no one making under $400,000 per year will pay a penny more in taxes ."

In [37]:
len(text), len(response[0]['summary_text']), len(response_ori[0]['summary_text'])

(448, 557, 253)

In [39]:
dataset['validation']

Dataset({
    features: ['document', 'summary'],
    num_rows: 5622
})

In [38]:
import time
import evaluate
import pandas as pd
import numpy as np

# tqdm library makes the loops show a smart progress meter.
from tqdm import tqdm
tqdm.pandas()

In [40]:
rouge = evaluate.load('rouge')

In [41]:
dialogues = dataset['validation'][0:50]['document']
human_baseline_summaries = dataset['validation'][0:50]['summary']

original_model_text = []
original_human_summaries = []
original_model_summaries = []
fine_tune_model_summaries = []

In [42]:
for idx, dialogue in enumerate(tqdm(dialogues)):
    prompt = f"""
Summarize:

{dialogue}
 """
    original_model_text.append(dialogue)
    original_human_summaries.append(human_baseline_summaries[idx])
    # summarize fine_tuned model
    response = summarizer(prompt)
    fine_tune_model_summaries.append(response[0]['summary_text'])
    # summarize original model
    response_ori = summarizer_ori(prompt)
    original_model_summaries.append(response_ori[0]['summary_text'])


100%|██████████| 50/50 [2:12:36<00:00, 159.12s/it]


In [44]:
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, fine_tune_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'fine_tune_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,fine_tune_model_summaries
0,– The Da Vinci Code has sold so many copies—th...,bestselling author Dan brown tops Oxfam's 'mos...,"– Dan Brown's latest novel, The Lost Symbol, s..."
1,– A major snafu has hit benefit payments to st...,the u.s. department of veterans affairs (va) a...,– It's been a rough year for veterans in the U...
2,– Yemen-based al-Qaeda in the Arabian Peninsul...,al Qaeda in the Arabian peninsula claims respo...,– The Charlie Hebdo magazine has sold out of i...
3,– Cambridge Analytica is calling it quits. The...,both Cambridge Analytica and its parent compan...,– The parent company of Cambridge Analytica is...
4,"– A lengthy report in the New York Times, base...",eavesdropping on foreign leaders has ignited a...,– Edward Snowden's revelations about the Natio...
5,– Don Juan de Oñate sought a city of gold when...,archaeologists have found evidence of a thrivi...,"– Archaeologists in Wichita, Kansas, say they'..."
6,– Another bad day for Anthony Weiner: Nancy Pe...,"""I've made some serious mistakes here but I'm ...",– Anthony Weiner is taking a break from his se...
7,– An augmented reality startup is being sued f...,lawsuit filed against magic leap alleges macho...,"– Magic Leap, the augmented-reality headset co..."
8,– The length of a man's index and ring fingers...,study: the ratio of the lengths of the index f...,– A new study suggests that the ratio of the l...
9,– A 71-year-old lawyer is suing United Airline...,a 71-year-old man is suing United Airlines and...,– A 71-year-old Houston man is suing United Ai...


In [ ]:
# https://medium.com/nlplanet/two-minutes-nlp-learn-the-rouge-metric-by-examples-f179cc285499

In [45]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)


In [46]:
print(original_model_results)

{'rouge1': 0.24661102484410957, 'rouge2': 0.08321107781417361, 'rougeL': 0.1489254718554845, 'rougeLsum': 0.14902202842624734}


In [47]:
fine_tune_model_results = rouge.compute(
    predictions=fine_tune_model_summaries,
    references=human_baseline_summaries[0:len(fine_tune_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [48]:
print(fine_tune_model_results)

{'rouge1': 0.42550844128521514, 'rouge2': 0.14595633205585168, 'rougeL': 0.21395292729139703, 'rougeLsum': 0.2130676574978273}


In [49]:
dialogues = dataset['validation'][:32]['document']

In [50]:
type(dialogues)

list

In [51]:
dialogues[0]

'Whether a sign of a good read; or a comment on the \'pulp\' nature of some genres of fiction, the Oxfam second-hand book charts have remained in The Da Vinci Code author\'s favour for the past four years. \n \n Dan Brown has topped Oxfam\'s \'most donated\' list again, his fourth consecutive year. Having sold more than 80 million copies of The Da Vinci Code and had all four of his novels on the New York Times bestseller list in the same week, it\'s hardly surprising that Brown\'s hefty tomes are being donated to charity by readers keen to make some room on their shelves. \n \n Another cult crime writer responsible to heavy-weight hardbacks, Stieg Larsson, is Oxfam\'s \'most sold\' author for the second time in a row. Both the \'most donated\' and \'most sold\' lists are dominated by crime fiction, trilogies and fantasy, with JK Rowling the only female author listed in either of the Top Fives. \n \n Click here or on "View Gallery" to see both charts in pictures ||||| A woman reads a co

In [52]:
summarizer = pipeline("summarization", model=fine_tune_model, tokenizer=tokenizer, device="cpu", batch_size=2 )


In [ ]:
response= summarizer(dialogues)

In [ ]:
len(response)

In [ ]:
summarizer_ori = pipeline("summarization", model=checkpoint, tokenizer=tokenizer,  device_map='auto')
response_ori = summarizer_ori(dialogues)

In [ ]:
len(response_ori)